In [1]:
# import libraries needed
import IPython
import pandas as pd
import numpy as np
from statistics import mode
import matplotlib.pyplot as plt  
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML
from pprint import pprint
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scipy import stats
import os

In [2]:
df = pd.read_csv('./data/diabetic_data.csv').sort_values(by=['patient_nbr'])
len(df)

101766

In [3]:
# drop rows

# mask = df['gender'] != 'Unknown/Invalid'
# mask &= df['diag_1'] != '?'  # TODO: only consider primary diag
# mask &= df['medical_specialty'] != '?'
# df = df[mask]
# print(len(df))

# remove all encounters that resulted in either discharge to a hospice or patient death
# to avoid biasing our analysis
df = df[~df['discharge_disposition_id'].isin([11, 13, 14, 19, 20, 21])]

print(len(df))

# consider only the first encounter for each patient
df = df.drop_duplicates(subset='patient_nbr', keep='first')

print(len(df))

99343
69990


# TODO
1. use top10 medical_specialty 
2. use diag 2, 3
3. use 24 features
6. check merge id

In [4]:
# # drop columns

# drop unrelated columns
df = df.drop(columns=["encounter_id", "patient_nbr"])  

# drop columns with too many missing values
df = df.drop(columns=["payer_code", "weight", "medical_specialty"]) 

# drop columns having same value in each row
df = df.drop(columns=["citoglipton", "examide"])

# drop diag_2, diag_3
df = df.drop(columns=["diag_2", "diag_3"])
df = df.drop(columns=["metformin",
    "repaglinide",
    "nateglinide",
    "chlorpropamide",
    "glimepiride",
    "acetohexamide",
    "glipizide",
    "glyburide",
    "tolbutamide",
    "pioglitazone",
    "rosiglitazone",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "insulin",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-rosiglitazone",
    "metformin-pioglitazone"
])

In [5]:
num_col = [
    'time_in_hospital',
    'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient',
    'number_emergency', 'number_inpatient', 'number_diagnoses',
]

# Removing skewnewss and kurtosis using log transformation if it is above a threshold value (2)

statdataframe = pd.DataFrame()
statdataframe['numeric_column'] = num_col
skew_before = []
skew_after = []

kurt_before = []
kurt_after = []

standard_deviation_before = []
standard_deviation_after = []

log_transform_needed = []

log_type = []

for i in num_col:
    skewval = df[i].skew()
    skew_before.append(skewval)
    
    kurtval = df[i].kurtosis()
    kurt_before.append(kurtval)
    
    sdval = df[i].std()
    standard_deviation_before.append(sdval)
    
    if (abs(skewval) >2) & (abs(kurtval) >2):
        log_transform_needed.append('Yes')
        
        if len(df[df[i] == 0])/len(df) <=0.02:
            log_type.append('log')
            skewvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).skew()
            skew_after.append(skewvalnew)
            
            kurtvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
        else:
            log_type.append('log1p')
            skewvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).skew()
            skew_after.append(skewvalnew)
        
            kurtvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
    else:
        log_type.append('NA')
        log_transform_needed.append('No')
        
        skew_after.append(skewval)
        kurt_after.append(kurtval)
        standard_deviation_after.append(sdval)

statdataframe['skew_before'] = skew_before
statdataframe['kurtosis_before'] = kurt_before
statdataframe['standard_deviation_before'] = standard_deviation_before
statdataframe['log_transform_needed'] = log_transform_needed
statdataframe['log_type'] = log_type
statdataframe['skew_after'] = skew_after
statdataframe['kurtosis_after'] = kurt_after
statdataframe['standard_deviation_after'] = standard_deviation_after

display(statdataframe)

# performing the log transformation for the columns determined to be needing it above.

for i in range(len(statdataframe)):
    if statdataframe['log_transform_needed'][i] == 'Yes':
        colname = str(statdataframe['numeric_column'][i])
        
        if statdataframe['log_type'][i] == 'log':
            df = df[df[colname] > 0]
            df[colname + "_log"] = np.log(df[colname])
            df = df.drop(columns=[colname])
            
        elif statdataframe['log_type'][i] == 'log1p':
            df = df[df[colname] >= 0]
            df[colname + "_log1p"] = np.log1p(df[colname])
            df = df.drop(columns=[colname])

,numeric_column,skew_before,kurtosis_before,standard_deviation_before,log_transform_needed,log_type,skew_after,kurtosis_after,standard_deviation_after
0,time_in_hospital,1.178414,1.005374,2.941530,No,NA,1.178414,1.005374,2.941530
1,num_lab_procedures,-0.222241,-0.275317,19.793030,No,NA,-0.222241,-0.275317,19.793030
2,num_procedures,1.253450,0.650776,1.739600,No,NA,1.253450,0.650776,1.739600
3,num_medications,1.406129,3.687561,8.270118,No,NA,1.406129,3.687561,8.270118
4,number_outpatient,9.336332,165.354879,1.136492,Yes,log1p,3.032207,9.790257,0.396187
5,number_emergency,18.372016,800.256668,0.593088,Yes,log1p,4.105493,20.298228,0.249578
6,number_inpatient,4.348797,32.198094,0.830361,Yes,log1p,2.087765,3.954927,0.390037
7,number_diagnoses,-0.753412,-0.316498,1.992313,No,NA,-0.753412,-0.316498,1.992313


In [6]:
# # number_inpatient_log1p, number_emergency_log1p, number_outpatient_log1p
# col = "number_outpatient"

# plt.figure()
# np.log1p(df[col]).plot.hist()
# # plt.show()
# plt.savefig('{}_log1p_hist.pdf'.format(col))

# plt.figure()
# df[col].plot.hist()
# plt.savefig('{}_hist.pdf'.format(col))
# # plt.show()


In [7]:
# merge ids with same meaning
def merge(df, col, same_ids):
    for ids in same_ids:
        for k in ids[1:]:
            df[col] = df[col].replace(k, ids[0])
    return df

df = merge(df, 'admission_type_id', [
    [1, 2, 7],  # emergence
    [5, 6, 8],  # not avaliable
])
df = merge(df, 'discharge_disposition_id', [
    [18, 25, 26],  # not avaliable
    [1, 6, 8],  # to home
    [2, 3, 4, 5],  # discharge to another hospital
    [10, 12, 15, 16, 17],  # discharge to outpatient
])
df = merge(df, 'admission_source_id', [
    [1, 2, 3], # Referral
    [4, 5, 6, 10, 22, 25], # from another hospital
    [9, 15, 17, 20, 21]  # not avaliable
])

In [8]:
# convert diagnosis information: reference from original paper
def in_range(x, bounds=None, sets=None):
    if bounds is None:
        bounds = []
    elif not isinstance(bounds[0], tuple):
        bounds = [bounds]
    
    if sets is None:
        sets = []
    elif not isinstance(sets, tuple):
        sets = [sets]
        
    for (l, r) in bounds:
        if x >= l and x < r:
            return True
    return x in sets

def convert_diag_func(x):
    ranges = {
        ((390, 460), 785),
        ((460, 520), 786),
        ((520, 580), 787),
        (None, 250),
        ((800, 1000), None),
        ((710, 740), None),
        ((580, 630), 788),
        ((140, 240), None),
    }
    if x == '?':
        return -1
    elif isinstance(x, str) and (x.startswith('V') or x.startswith('E')):
        return 0

    for i, (bounds, sets) in enumerate(ranges):
        if in_range(int(float(x)), bounds, sets):
            return i+1
    else:
        return 0
        
df['diag_1'] = df['diag_1'].apply(convert_diag_func)
# df['diag_2'] = df['diag_2'].apply(convert_diag_func)
# df['diag_3'] = df['diag_3'].apply(convert_diag_func)

In [9]:
# df = df[df['diag_1']  == 4]

In [10]:
def show_crosstab(col):
    grouped = df.groupby([col, 'readmitted']).size().to_frame().reset_index()
    for c, n in df.groupby([col]).size().iteritems():
        grouped.loc[grouped[col] == c, 0] /= n
    grouped[0] = grouped[0].apply(lambda x: "{:.2%}".format(x))
    crosstab = pd.crosstab(grouped[col], grouped['readmitted'], values=grouped[0], aggfunc=lambda x: x)
    display(crosstab)

show_crosstab('max_glu_serum')
show_crosstab('A1Cresult')
# show_crosstab('medical_specialty')

readmitted,<30,>30,NO
max_glu_serum,,,
>200,7.94%,29.24%,62.82%
>300,8.67%,31.36%,59.97%
None,7.34%,26.83%,65.82%
Norm,7.34%,25.70%,66.96%


readmitted,<30,>30,NO
A1Cresult,,,
>7,6.74%,25.28%,67.97%
>8,6.39%,26.02%,67.59%
None,7.52%,27.23%,65.25%
Norm,7.02%,24.14%,68.84%


In [11]:
def encode(df, col, d):
    df[col] = df[col].apply(lambda x: d[x])
    return df

def encode_with_none(df, col, d):
    df[col+'_is_none'] = df[col] == 'None'
    d['None'] = -1
    return encode(df, col, d)

df['age'] = df['age'].apply(lambda x: int(x[1]) * 10 + 5)  # encode [0-10) - [90, 100] to 0 - 9
df = encode(df, 'readmitted', {"NO": 0, ">30": 1, "<30": 2})
df = encode_with_none(df, 'max_glu_serum', {">300": 2, ">200": 1, "Norm": 0})
df = encode_with_none(df, 'A1Cresult', {">8": 2, ">7": 1, "Norm": 0})

df = pd.get_dummies(df, drop_first = False, columns=[
    'race', 
    'gender', 
    'admission_type_id', 
    'discharge_disposition_id',
    'admission_source_id', 
    'diag_1', # 'diag_2', 'diag_3',
    'change', 
    'diabetesMed',
#     'medical_specialty',
])

In [12]:
df['number_outpatient_log1p']

4267      0.000000
5827      0.000000
67608     0.000000
17494     0.000000
2270      0.000000
14180     0.000000
18234     0.000000
15848     0.000000
61382     1.386294
2279      0.000000
7866      0.000000
25911     0.000000
1083      0.000000
2001      0.000000
11049     0.000000
2484      0.000000
17342     0.000000
23541     0.000000
4407      0.000000
7038      0.000000
2005      0.000000
10001     0.000000
21483     0.000000
3294      0.000000
22342     0.000000
36317     0.000000
4333      0.000000
18558     0.000000
36720     0.000000
18390     0.000000
            ...   
101755    0.000000
99544     0.000000
96047     0.000000
97982     0.000000
96274     0.000000
98897     0.000000
99798     0.000000
99556     0.000000
91913     0.000000
91774     0.000000
93108     0.000000
93052     0.000000
93050     0.000000
96345     0.000000
93742     0.000000
95669     0.000000
100090    0.693147
90717     0.000000
92165     0.000000
95032     0.000000
94231     0.000000
90933     0.

In [13]:
df.head(18).T

,4267,5827,67608,17494,2270,14180,18234,15848,61382,2279,7866,25911,1083,2001,11049,2484,17342,23541
age,55,55,85,85,35,65,65,45,75,75,65,75,65,75,85,65,45,85
time_in_hospital,8,2,4,3,5,10,9,2,14,12,8,1,2,7,7,4,1,7
num_lab_procedures,77,49,68,46,49,54,52,50,21,47,57,31,15,27,77,47,35,51
num_procedures,6,1,2,0,0,2,1,5,0,2,6,1,0,3,0,4,5,0
num_medications,33,11,23,20,5,19,16,13,15,18,31,9,14,16,12,16,13,13
number_diagnoses,8,3,9,9,3,9,9,9,7,9,9,7,9,9,9,7,8,9
max_glu_serum,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A1Cresult,-1,-1,1,2,-1,-1,-1,-1,-1,0,2,-1,-1,2,-1,-1,-1,-1
readmitted,2,0,0,0,0,1,0,1,1,0,0,0,1,1,0,0,0,0
number_outpatient_log1p,0,0,0,0,0,0,0,0,1.38629,0,0,0,0,0,0,0,0,0


In [14]:
df.columns 

Index(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'readmitted', 'number_outpatient_log1p', 'number_emergency_log1p',
       'number_inpatient_log1p', 'max_glu_serum_is_none', 'A1Cresult_is_none',
       'race_?', 'race_AfricanAmerican', 'race_Asian', 'race_Caucasian',
       'race_Hispanic', 'race_Other', 'gender_Female', 'gender_Male',
       'gender_Unknown/Invalid', 'admission_type_id_1', 'admission_type_id_3',
       'admission_type_id_4', 'admission_type_id_5',
       'discharge_disposition_id_1', 'discharge_disposition_id_2',
       'discharge_disposition_id_7', 'discharge_disposition_id_9',
       'discharge_disposition_id_10', 'discharge_disposition_id_18',
       'discharge_disposition_id_22', 'discharge_disposition_id_23',
       'discharge_disposition_id_24', 'discharge_disposition_id_27',
       'discharge_disposition_id_28', 'admission_source_id_1',
       'admission_

# Train and test

In [82]:
from sklearn.metrics import precision_recall_fscore_support

def f1_macro(y_test, y_pred, vervose=False):
    p, r, f1, s = precision_recall_fscore_support(y_test, y_pred)

    p_macro = np.mean(p)
    r_macro = np.mean(r)
    f1_macro = 2 * p_macro * r_macro / (p_macro + r_macro)
    
    return f1_macro

def f1_macro_all(y_test, y_pred, vervose=False):
    p, r, f1, s = precision_recall_fscore_support(y_test, y_pred)

    p_macro = np.mean(p)
    r_macro = np.mean(r)
    f1_macro = 2 * p_macro * r_macro / (p_macro + r_macro)

    return np.concatenate([f1, np.array([f1_macro])], axis=0)

In [93]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=0)
f1_results = []
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    X_train, y_train = SMOTE(random_state=20).fit_sample(X_train, y_train)

    weight_per_class = np.sqrt(y.shape[0] / (df['readmitted'].value_counts()))
    w_train = pd.Series(y_train).map(weight_per_class).values

#     xg_train = xgb.DMatrix(X_train, label=y_train, weight=w_train)
    xg_train = xgb.DMatrix(X_train, label=y_train)
    xg_test = xgb.DMatrix(X_test, label=y_test)

    def f1_macro_monitor(preds, dtrain):
        labels = dtrain.get_label()
        return "f1_macro", f1_macro(labels, preds)

    os.environ["CUDA_VISIBLE_DEVICES"] = "4"
    param = {
        'booster': 'dart',
        "objective": 'multi:softmax',
        "num_class": 3,
        "silent": 1,

        "gpu_id": 0,
        "tree_method": "gpu_hist",

        'lambda': 1,
        'alpha': 0,

        "eta": 1,
        "max_depth": 6,
    }

    num_round = 6

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    bst = xgb.train(param, xg_train, num_round, watchlist, feval=f1_macro_monitor)
    
    f1 = f1_macro_all(y_test, bst.predict(xg_test))
    f1_results.append(f1)
    print(f1)
np.mean(np.array(f1_results), axis=0)

[0]	train-merror:0.38607	test-merror:0.335048	train-f1_macro:0.604607	test-f1_macro:0.438979
[1]	train-merror:0.360185	test-merror:0.331476	train-f1_macro:0.631986	test-f1_macro:0.43108
[2]	train-merror:0.343204	test-merror:0.329619	train-f1_macro:0.649699	test-f1_macro:0.428953
[3]	train-merror:0.335749	test-merror:0.329761	train-f1_macro:0.65783	test-f1_macro:0.42504
[4]	train-merror:0.327359	test-merror:0.328761	train-f1_macro:0.66707	test-f1_macro:0.432982
[5]	train-merror:0.322086	test-merror:0.32919	train-f1_macro:0.673255	test-f1_macro:0.431469
[0.79737353 0.28211971 0.04982206 0.43146903]
[0]	train-merror:0.387463	test-merror:0.346192	train-f1_macro:0.60487	test-f1_macro:0.410435
[1]	train-merror:0.354797	test-merror:0.337048	train-f1_macro:0.637685	test-f1_macro:0.408512
[2]	train-merror:0.34303	test-merror:0.336191	train-f1_macro:0.650328	test-f1_macro:0.401837
[3]	train-merror:0.334372	test-merror:0.332762	train-f1_macro:0.659917	test-f1_macro:0.402754
[4]	train-merror:0.326

array([0.79409282, 0.26250017, 0.0437148 , 0.41801309])

In [58]:
y = df['readmitted'].values # .astype(int)
X = df.drop(columns='readmitted').values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# X_train, y_train = SMOTE(random_state=20).fit_sample(X_train, y_train)

weight_per_class = np.sqrt(y.shape[0] / (df['readmitted'].value_counts()))
w_train = pd.Series(y_train).map(weight_per_class).values

print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

xg_train = xgb.DMatrix(X_train, label=y_train, weight=w_train)
# xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

def f1_macro_monitor(preds, dtrain):
    labels = dtrain.get_label()
    return "f1_macro", f1_macro(labels, preds)

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
param = {
    'booster': 'dart',
    "objective": 'multi:softmax',
    "num_class": 3,
    "silent": 1,
    
    "gpu_id": 0,
    "tree_method": "gpu_hist",
    
    'lambda': 1,
    'alpha': 0,
    
    "eta": 0.3,
    "max_depth": 3,
}

num_round = 50

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
bst = xgb.train(param, xg_train, num_round, watchlist, feval=f1_macro_monitor)


2    36769
1    36769
0    36769
dtype: int64
0    9251
1    3713
2    1034
dtype: int64
[0]	train-merror:0.424755	test-merror:0.742606	train-f1_macro:0.456928	test-f1_macro:0.385188
[1]	train-merror:0.409923	test-merror:0.672525	train-f1_macro:0.485456	test-f1_macro:0.394428
[2]	train-merror:0.401866	test-merror:0.645949	train-f1_macro:0.508269	test-f1_macro:0.402822
[3]	train-merror:0.402047	test-merror:0.645378	train-f1_macro:0.509716	test-f1_macro:0.405473
[4]	train-merror:0.396977	test-merror:0.647307	train-f1_macro:0.498542	test-f1_macro:0.40327
[5]	train-merror:0.387312	test-merror:0.614374	train-f1_macro:0.511644	test-f1_macro:0.40991
[6]	train-merror:0.377729	test-merror:0.594013	train-f1_macro:0.52068	test-f1_macro:0.41416
[7]	train-merror:0.371863	test-merror:0.570367	train-f1_macro:0.529445	test-f1_macro:0.41539
[8]	train-merror:0.371351	test-merror:0.573296	train-f1_macro:0.527769	test-f1_macro:0.413796
[9]	train-merror:0.364204	test-merror:0.543935	train-f1_macro:0.54007	

In [16]:
class_weight = {k: v for (k, v) in weight_per_class.items()}

In [70]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
f1_results = []
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    dte = DecisionTreeClassifier(max_depth=11, criterion = "entropy", min_samples_split=10, class_weight=class_weight)
    dte.fit(X_train, y_train)
    y_pred = dte.predict(X_test)
    
    f1_results.append(f1_macro(y_test, y_pred))
print(np.mean(f1_results))

0.4200192464589966


In [53]:
from sklearn.ensemble import RandomForestClassifier

f1_results = []
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    forrest = RandomForestClassifier(n_estimators=15, max_depth=15, criterion = "gini", min_samples_split=10, class_weight=class_weight)
    forrest.fit(X_train, y_train)
    y_pred = forrest.predict(X_test)

    f1_results.append(f1_macro(y_test, y_pred))
print(np.mean(f1_results))

0.4200536592633847


In [54]:
from sklearn.linear_model import LogisticRegression
f1_results = []
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    logreg = LogisticRegression(fit_intercept=True, penalty='l1')#, class_weight=class_weight)
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    f1_results.append(f1_macro(y_test, y_pred))
print(np.mean(f1_results))

/home/liubin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/liubin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/liubin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/liubin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/liubin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classif

0.4161243401385362
